## <center>欢迎来到LightGBM的世界</center>

### Mac下的编译安装


#### 命令如下：

brew install cmake

brew install gcc --without-multilib


git clone --recursive https://github.com/Microsoft/LightGBM

cd LightGBM

export CXX=g++-7 CC=gcc-7

mkdir build

cd build

cmake ..

make -j4

pip3 install lightgbm


#### 温暖提示：

cd python-package

sudo python setup.py install 

会依然提示CMake没有安装的问题，至今不知道是为什么？

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [37]:
train= pd.read_csv('multiclass.train', header=None, sep = '\t')
test= pd.read_csv('multiclass.test', header=None, sep = '\t')
num_train = train.shape[0]
kfolds = 0.9


X_train = train.ix[:int(kfolds * num_train),1:]
y_train = train.ix[:int(kfolds * num_train),0]

X_val = train.ix[int(kfolds * num_train):,1:]
y_val = train.ix[int(kfolds * num_train):,0]

X_test = test.ix[:,1:]
y_test = test.ix[:,0]

lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_val, y_val)

num_class = y_train.unique().max()+1


(700,)

In [30]:
params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'metric': {'multi_error'},
            'num_class': num_class,
            'num_leaves': 80,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 4,
            'verbose': 0,
            #'device':'gpu'
        }

In [ ]:
min_merror = float('Inf')
best_params = {}
for learning_rate in [0.1]:
    for num_boost_round in range(30, 31):
        for max_depth in range(6, 8):
            params['learning_rate'] = learning_rate
            params['max_depth'] = max_depth
            cv_results = lgb.cv(
                            params,
                            lgb_train,
                            num_boost_round=num_boost_round,
                            seed=42,
                            nfold=3,
                            metrics=['multi_error'],
                            early_stopping_rounds=3
                          )
            mean_merror = pd.Series(cv_results['multi_error-mean']).min()
            boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()
            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['learning_rate'] = learning_rate
                best_params['num_boost_round'] = boost_rounds
                best_params['max_depth'] = max_depth

params['learning_rate'] = best_params['learning_rate']
params['max_depth'] = best_params['max_depth']
num_round = best_params['num_boost_round']

# using lgb_test as valid_sets
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=num_round,
                valid_sets=lgb_test,
                early_stopping_rounds=3)

[1]	valid_0's multi_error: 0.772857
Training until validation scores don't improve for 3 rounds.
[2]	valid_0's multi_error: 0.75
[3]	valid_0's multi_error: 0.734286
[4]	valid_0's multi_error: 0.72
[5]	valid_0's multi_error: 0.702857
[6]	valid_0's multi_error: 0.711429
[7]	valid_0's multi_error: 0.711429
[8]	valid_0's multi_error: 0.692857
[9]	valid_0's multi_error: 0.691429
[10]	valid_0's multi_error: 0.704286
[11]	valid_0's multi_error: 0.687143
[12]	valid_0's multi_error: 0.685714
[13]	valid_0's multi_error: 0.672857
[14]	valid_0's multi_error: 0.665714
[15]	valid_0's multi_error: 0.665714
[16]	valid_0's multi_error: 0.657143


In [58]:
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
acc = accuracy_score(y_val, np.argmax(y_pred, axis=1))
print(acc)

0.351428571429


In [44]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
acc = accuracy_score(y_test, np.argmax(y_pred, axis=1))
print(acc)

0.376


In [46]:
gbm.save_model('model.001')

In [55]:
params['metric'] = 'auc'
bst=lgb.cv(params, lgb_train, num_boost_round=10, nfold=3, early_stopping_rounds=3)
print(bst)
estimators = lgb.train(params, lgb_train,num_boost_round=len(bst['auc-mean']))

y_pred = estimators.predict(X_)

{'auc-mean': [0.41206949953192146], 'auc-stdv': [0.0039421307840286929]}
